In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data = pd.read_csv("datasets_hi4/train-data.csv",sep=";")

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3173346 entries, 0 to 3173345
Data columns (total 19 columns):
 #   Column                          Dtype 
---  ------                          ----- 
 0   index                           int64 
 1   id_product                      int64 
 2   Region                          object
 3   Country                         object
 4   Site                            object
 5   Operations                      object
 6   Zone                            object
 7   Cluster                         object
 8   Reference proxy                 object
 9   Product  Line proxy             object
 10  Division proxy                  object
 11  Customer Persona proxy          object
 12  Strategic Product Family proxy  object
 13  Product Life cycel status       object
 14  Date                            object
 15  Month 1                         object
 16  Month 2                         object
 17  Month 3                         object
 18  Mo

In [7]:
data.describe()

,index,id_product
count,3.173346e+06,3.173346e+06
mean,1.101597e+06,1.223989e+05
std,6.362905e+05,7.068357e+04
min,0.000000e+00,0.000000e+00
25%,5.506670e+05,6.115800e+04
50%,1.101594e+06,1.223590e+05
75%,1.652761e+06,1.836168e+05
max,2.203712e+06,2.448560e+05


In [9]:
data.set_index("index")

,index,id_product,Region,Country,Site,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Date,Month 1,Month 2,Month 3,Month 4
0,645874,156160,EUROPE,DE,NL_DC_Venray,Europe Operations,DACH,Germany,reference-13523,Product Line-4,Division-3,Customer Segmentation-3,Strategic Product Family-12,NaN,may-aug 2021,0,0,0,0
1,469488,224631,CHINA,CN,CN_DC_Shanghai,China Operations,China & HK,China,reference-12778,Product Line-4,Division-3,Customer Segmentation-14,Strategic Product Family-9,NaN,jan-apr 2021,0,1,0,0
2,348904,104047,EUROPE,GB,NL_DC_HLD,Europe Operations,UK and Ireland,United Kingdom,reference-3513,Product Line-3,Division-2,Customer Segmentation-23,Strategic Product Family-7,NaN,jan-apr 2021,0,2,0,0
3,1725822,11823,EAJP,AU,AU_DC_Perth,International Operations,Pacific,Australia,reference-672,Product Line-1,Division-1,Customer Segmentation-11,Strategic Product Family-3,NaN,jan-apr 2023,0,0,0,0
4,404781,159924,EUROPE,ES,NL_DC_HLD,Europe Operations,Iberia,Spain,reference-3496,Product Line-3,Division-2,Customer Segmentation-14,Strategic Product Family-7,NaN,jan-apr 2021,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3173341,1285020,60735,NAM,US,US_DC_Mechanicsburg,North America Operations,US,USA,reference-6024,Product Line-4,Division-3,Customer Segmentation-4,Strategic Product Family-12,EOL,may-aug 2022,0,0,0,0
3173342,1527076,57934,NAM,US,US_DC_Athens,North America Operations,US,USA,reference-5672,Product Line-4,Division-3,Customer Segmentation-43,Strategic Product Family-9,NaN,sep-dec 2022,0,0,0,0
3173343,306473,61616,NAM,US,US_DC_Mechanicsburg,North America Operations,US,USA,reference-6241,Product Line-4,Division-3,Customer Segmentation-46,Strategic Product Family-12,NaN,jan-apr 2021,0,0,0,0
3173344,205624,205624,EUROPE,FR,FR_DC_Evreux,France Operations,France,France,reference-11174,Product Line-4,Division-3,Customer Segmentation-1,Strategic Product Family-9,NaN,sep-dec 2020,0,1,0,0
